In [1]:
import random
import math
import matplotlib.pyplot as plt
import time
import numpy as np
from point import Point

Same pool of trials for all participants (static), with different shuffling (dynamic).

---

# Static Layout Pool
This refers to the generation of the layout pool, which will is the same for ALL participants.

## Generation

- Given N trials, we produce  N/2 pairs of layouts.
    - Between these pairs, we keep the same coordinates for the points, and the same order of navigation.
    - One of these pairs is the no-boundary-show, the other is the boundary-shown. The layout is rotated between these two.
    - Per layout, we have 5 objects and an ending location. The first object is also the starting location and might be asked.
    - We do not want any 2 objects to be too close to each other.
    - We do not want any path to be too short. We do not have an upper bound on the travel distance.
    - We want to to make sure that there is at least one object on both sides.

In [2]:
N_POINTS_LAYOUT = 6 # 5 objects, one of which is the starting location, and an ending location
N_LAYOUTS = 50 # generate this many layouts. each layout produces a pair (visible/invisible boundary)

# these are currently arbitrary units that we can scale up and down as needed
# w=5, l=10 represents a 1 by 2 rectangle room
WIDTH = 5 
LENGTH = 10

# this is the minimum distance any two objects (not a path) can have. 
# used to prevent objects appearing too close and confusing participants
MIN_DIST_OBJECTS = 1.5

# this is the minimum path distance. no subsequent object should appear closer than this.
# greater than MIN_DIST_OBJECTS because objects appearing too close also means they are visited too soon and not enough time to learn that object
MIN_DIST_PATH = 2.3

In [5]:
def generate_random_layout():
    return [Point.random_rect(WIDTH, LENGTH) for p in range(N_POINTS_LAYOUT)]

def is_valid_layout(lay):
    
    # check all distance constraints, (dist between objects and path distances)
    for i in range(len(lay)):
        for j in range(i):
            # if these points are adjacent, check for path distance condition, otherwise object distance condition
            lower_bound = MIN_DIST_PATH if j == i - 1 else MIN_DIST_OBJECTS
            if lay[i].dist(lay[j]) <= lower_bound:
                return False
    
    # check that there is at least one object on each side
    # note that this checks for objects ONLY (that is; not the last point which is the ending/task location)
    objects_on_lower_side = sum([1 if l.y < LENGTH / 2 else 0 for l in lay[:N_POINTS_LAYOUT-1]])
    if not ( 1 <= objects_on_lower_side < N_POINTS_LAYOUT-1):
        # either 0 or N_POINTS_LAYOUT-1 on this side, which means the reverse for the other side. not valid!
        return False
            
    return True

def generate_valid_layout():
    candidate = generate_random_layout()
    c = 0
    while not is_valid_layout(candidate):
        c += 1
        if c == 1e5:
            print("can not find valid layout!")
            return None
        candidate = generate_random_layout()
    return candidate

In [15]:
layouts = []
for l in range(N_LAYOUTS):
    layouts.append(generate_valid_layout())

In [16]:
print(layouts)

[[P(1.23, 9.32), P(4.6, 7.5), P(0.41, 2.25), P(2.59, 1.01), P(3.7, 9.45), P(3.03, 5.01)], [P(0.02, 6.27), P(3.96, 3.23), P(3.07, 7.64), P(3.23, 0.96), P(4.02, 5.28), P(4.67, 9.28)], [P(4.67, 3.0), P(0.22, 5.14), P(3.16, 0.1), P(0.62, 8.39), P(1.14, 2.46), P(3.83, 9.65)], [P(4.04, 2.22), P(0.08, 7.43), P(4.06, 7.17), P(1.06, 3.18), P(3.54, 0.29), P(2.13, 8.45)], [P(4.41, 5.4), P(2.14, 0.54), P(0.61, 5.07), P(3.67, 1.27), P(0.15, 9.3), P(2.32, 2.43)], [P(0.95, 4.24), P(0.45, 7.9), P(2.48, 5.24), P(3.47, 9.04), P(1.35, 2.32), P(4.55, 4.01)], [P(2.14, 0.47), P(1.16, 2.93), P(2.99, 6.2), P(3.09, 9.82), P(3.51, 2.54), P(1.18, 5.65)], [P(3.17, 4.09), P(1.12, 1.81), P(1.46, 4.74), P(4.78, 1.4), P(0.33, 7.56), P(4.19, 5.41)], [P(2.98, 4.04), P(0.02, 9.05), P(0.65, 2.44), P(1.09, 4.74), P(2.02, 9.61), P(4.13, 0.56)], [P(4.73, 5.58), P(0.2, 9.04), P(3.92, 8.37), P(1.08, 6.49), P(0.98, 1.01), P(4.32, 0.6)], [P(1.32, 6.71), P(2.56, 2.85), P(1.13, 9.87), P(0.6, 0.29), P(4.84, 6.45), P(0.2, 3.32)], [

## Checks
- We want to check for the following distributions:
    - Pointing ground-truths -> compare those in the same side and across the boundary.
    - How much they have traveled
    - Distances between any two objects

# Dynamic Trial Shuffling
- We shuffle the N trials such that the boundary visibility alternates each time.

## Checks

- At least K = 5 trials between trials of the same pairing.
- We make sure that approx. half the pairs happen in A->B, and the other half happens in B->A
- We make sure that between adjacent pairs, the ending location and starting location is always opposite.
- We want to check the number of objects within the same cells across a sliding window of 5 trials.

# Objects
- 5 different shapes (circle, cube, cylinder, pyramid, star)
- 5 different colors from a color-blind-friendly palette.
-> 25 different combinations.

Any object generated must not have been in the last 10 shown object.